In [4]:
import numpy as np
import bokeh as bk
from bokeh.io import push_notebook
import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.io import push_notebook
from ipywidgets import interact
import ipywidgets as widgets
bkp.output_notebook()

Loading BokehJS ...

# Solidification week 4 homework

## Mikko Karkkainen 11740378

### Problem 1.

Consider a casting solidifying in a sand mold whose thermal properties (km, cm) are known. Develop an analytical solution for the temporal evolution of temperature in the casting (cooling curve) assuming the casting is poured at some superheating temperature

### Solution

To be able to use an analytic solution for this problem, we must use lump analysis and the assumption of a low (<0.1) biot number. We get the solution:

$$ T(t) = T_0+(T_i-T_0)exp\Bigg( \frac{\overline h At}{v \rho c (1-(b/c) \Delta H_f)} \Bigg) $$

where $ T_i $ is the initial temperature, $ T_0 $ is the ambient temperature, $ \Delta H_f $ is the enthalpy of fusion, $ \overline h $ is the average heat transfer coefficient for convection and $ b $ is calculated from the evolution of the solid fraction as a function of temperature$ \big( f_s(T)=a+bT \big)$

However, by using a time-stepping method(numerical), we could estimate the cooling curve using the sand mold properties using the equation:

$$ T^{new} = T^{old} - \frac{A}{v \rho c} \sqrt{\frac {k_m \rho_m c_m}{\pi t^{new}}}\Big( T^{old} - T_m\Big) \Delta t + \frac{\Delta H_{f}}{c} \Delta f_s $$ 


## Problem 2.

2. An Al-7% Si alloy is poured in a mold to produce a casting having a volume of 12 cm3 and a cooling area of 26 cm2. Consider two different mold materials: silica sand and cast iron. Assuming the alloy is poured at 715 ºC, calculate and present graphically the cooling curves corresponding to the two different mold materials using a time stepping analysis.  Also, calculate the heat transfer coefficients for the cases and present their time dependency in graphical format.


## Solution:

We will use the Chvorinov equation presented in problem 1 for this analysis. 

Information given in problem:
$$v= 1.2 \cdot 10^{-5} m^3,
A=0.0026 m^2, 
T^{i}=988.15 K $$

Selected properties of Al-Si (from textbook and phase diagram of Al-Si):

$$
T_L=890 K,
T_S=850 K,
\Delta T_0= 40 K.
 \rho = 2350 ,
c_s = 1170 J kg^{-1}K^{-1}, \Delta H_f = 3.91 \cdot 10^5 J kg^{-1} $$

Selected properties of grey cast iron (from textbook):

$$
 \rho = 7050 kg m^{-3} ,
c_s = 794 J kg^{-1}K^{-1},
k_s = 40 $$

Selected properties of silica sand (from textbook)

$$
 \rho = 1600 kg m^{-3} ,
c_s = 1170 J kg^{-1}K^{-1},
k_s = 0.52 $$

We will use the equation from example 6.1:


$$ T^{n+1} = T^{n} - \frac{A}{v \rho} \frac{T^n-T_m}{\Delta H_f/\Delta T_0 + c} \sqrt{\frac{k_m \rho_m c_m}{\pi t^{n+1}}} \Delta t $$


In [5]:
Delta_t=1
T_i=988.15
Delta_T_0=40
v=1.2*10**(-5)
A=0.0026
Delta_H_f=3.91*10**5
rho=2350
c=1170
T_L=890
T_S=850

#cast iron mold:

rho_fe=7050
k_fe=40
c_fe=794


#silica sand mold:

rho_si=1600
k_si=0.52
c_si=1170


#simulation parameters:

step_count=200
temp_values_fe=np.empty(step_count)
temp_values_fe[0]=T_i

temp_values_si=np.empty(step_count)
temp_values_si[0]=T_i

h_values_fe=np.zeros(step_count)
h_values_si=np.zeros(step_count)

def calc_temp(rho_m,k_m,c_m,temp_values):
    for i in range(1,step_count):
        if (temp_values[i-1]>=T_L or temp_values[i-1]<=T_S):
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-298.15)/(rho*(c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t
        else:
            temp_values[i]=temp_values[i-1]-A/v*(temp_values[i-1]-298.15)/(rho*(Delta_H_f/Delta_T_0 + c))*np.sqrt((k_m*rho_m*c_m/(np.pi*(Delta_t*i))))*Delta_t

def calc_h(rho_m,k_m,c_m,h_values):
    for i in range(1,step_count):
        h_values[i]=np.sqrt((k_m*rho_m*c_m)/(np.pi*Delta_t*i))

In [6]:
calc_temp(rho_fe,k_fe,c_fe,temp_values_fe)

In [7]:
calc_temp(rho_si,k_si,c_si,temp_values_si)

In [8]:
calc_h(rho_fe,k_fe,c_fe,h_values_fe)

In [9]:
calc_h(rho_si,k_si,c_si,h_values_si)

In [10]:
f=bkp.figure(title="Cooling curve of Al-Si casting", x_axis_label="time (s)", y_axis_label="Temperature (K)")
time_series=np.linspace(0,step_count*Delta_t,step_count)
time_series_h=np.linspace(Delta_t,step_count*Delta_t,step_count)

f.line(time_series,temp_values_fe, legend="Cast Iron mold")
f.line(time_series,temp_values_si, color="red", legend="Silica sand mold")


bkp.show(f)

In [11]:
f=bkp.figure(title="Heat transfer co-efficient of casting", x_axis_label="time (s)", y_axis_label="h (J/(m^2 K s))")
time_series=np.linspace(0,step_count*Delta_t,step_count)

f.line(time_series_h,h_values_fe[1::], legend="Cast Iron mold")
f.line(time_series_h,h_values_si[1::], color="red", legend="Silica sand mold")


bkp.show(f)

## Problem 3.

Calculate the solidification time of an aluminum cube of volume 0.001m3 poured into a silica sand mold
from a superheating temperature of 680°C. Use a time-stepping analysis (march forward in time)
assuming resistance in the mold.

## Solution:




Time to use time-stepping Chvorinov again. We will use the mold parameters from problem 2. For pure aluminum, the necessary parameters in SI units are:

$$ \Delta H_f = 3.91 \cdot 10^{5}, \, k_s = 213, \, c_S=1170, \, \rho =2700, \, T_L=933.15 K, \, \Delta T_0 = 0 $$

Let's assume that the aluminum solidification takes place over the temperature range of $[T_L, T_L-5] $

In [12]:
Delta_H_f=3.91*10**5
k=213
c=1170
rho=2700
T_L=933.15
T_S=933.15-5
Delta_T_0=5
v=0.001
A=0.06
T_i=953.15
Delta_t=1

step_count=1000

al_temp_values=np.linspace(0,step_count,step_count)
al_temp_values[0]=T_i

calc_temp(rho_si,k_si,c_si,al_temp_values)

In [13]:
f=bkp.figure(title="Cooling curve of Al cube", x_axis_label="time (s)", y_axis_label="Temperature (K)")
time_series=np.linspace(0,step_count*Delta_t,step_count)

f.line(time_series,al_temp_values, color="red", legend="Silica sand mold")


bkp.show(f)

## Problem 4

Consider a rod (1D casting) of length 120 mm. poured into a sand mold. The casting material is
eutectic Al-Si alloy, poured at 50oC superheating above the eutectic temperature. Assume that the
solidification interval is 8oC. Calculate the solidification time of the casting using a 1D Finite Difference
Method (FDM) scheme. Plot the cooling curves in the middle of the casting as well as at a position 20
mm from the edge of the casting.

## Solution:

For 1D explicit finite difference:

$$ T^{new} = T^{old} + \frac{Fo}{1-\frac{\Delta H_f}{c} \frac{\partial f_s}{\partial T}}\Big( T_E^{old} - 2T^{old} + T^{old}_W \Big) $$

where $Fo$ is:

$$ Fo= \frac{\alpha(\Delta t)}{(\Delta x)^2} $$

The stability criterion for 1D is given by:

$$ Fo \leq \frac{1}{2}\Big( \frac{h \Delta s}{k} +1 \Big)^{-1} $$

Assuming that $ f_s = 0 $ at $ T_L $ and that $ f_s =1$ at $ T_L-8 $, we get:

$$ \frac{\partial f_s}{\partial T} = -\frac{1}{8} $$

But only when $ T_L-8 < T < T_L $

From the text book, we get the following equation for the node at the mold-casting border:

$$ T^{new} = \frac{2Fo}{a} \big( T_E^{old}+bT_{mold} \big) + \Big[ 1- \frac{2Fo}{a}(b+1) \Big] T^{old}  $$

where

$$ a= 1+\frac{\Delta H_f}{c \Delta T_0}$$

and

$$ b= \frac{h \Delta x}{k} $$

For the center of the casting, a zero flux condition is assumed

In [19]:
# 2D array for temperature values in time:
Delta_x=0.060/9

T_i=900.35
Delta_T_0=8
Delta_H_f=3.91*10**5
rho=2350
c=1170
T_L=850.35
T_S=842.35
h=56
k=213

x_step_count=10
t_step_count=8000
a=1+Delta_H_f/(c*Delta_T_0)
b=h*Delta_x/k

alpha=k/(rho*c)
Fo=alpha*Delta_t/(Delta_x**2)

Delta_t=Delta_x**2/(2*alpha)*((h*Delta_x)/k+1)**(-1)

T_values=np.empty([t_step_count,x_step_count])
T_values=T_values.reshape(-1)

T_values.fill(T_i)

print(T_values.shape)

for i in range(1,len(T_values)):
    if (T_values[i-x_step_count]>T_L or T_values[i-x_step_count]<T_S):
        a=1
    else:
        a=1+Delta_H_f/(c*Delta_T_0)
    
    if (np.remainder(i,x_step_count)==0):
        T_values[i]=2*Fo/a*(T_values[i+1-x_step_count]+b*298.15)+(1-2*Fo/a*(b+1))*T_values[i-(x_step_count)]  
    elif (np.remainder(i+1,x_step_count)==0):
        T_values[i]=T_values[i-1]
    else:
        T_values[i]=T_values[i-(x_step_count)]+Fo/a*(T_values[i-(x_step_count-1)]-2*T_values[i-(x_step_count)]+T_values[i-(x_step_count+1)])

T_values=T_values.reshape([-1,x_step_count])

time_series=np.linspace(0,Delta_t*t_step_count,t_step_count)


(80000,)


In [17]:
f=bkp.figure(title="1D fdm Al-Si rod",x_axis_label="time (s)", y_axis_label="Temperature (K)")
f.line(time_series, T_values[::,x_step_count-1] ,legend="Middle of casting")
f.line(time_series, T_values[::,3] ,legend="20 mm of edge", color="red")
bkp.show(f)

In [607]:
print(0.02/Delta_x)

3.0000000000000004
